In [13]:
import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

In [14]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    base_File = f"{baseFile}/u{fold_Number}.base"
    test_File = f"{baseFile}/u{fold_Number}.test"
    base_Data = pd.read_csv(base_File, sep="\t", header=None, names=rnmColData)
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    base_Data = base_Data.drop(columns=["timestamp"])
    test_Data = test_Data.drop(columns=["timestamp"])
    return base_Data, test_Data

In [15]:
call_base = "../ml-100k"
basedata, testdata = load_Dataset(call_base, 1)
testdata


,user_id,item_id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3
...,...,...,...
19995,458,648,4
19996,458,1101,4
19997,459,934,3
19998,460,10,3


In [16]:
# 

In [17]:
# [1:463, 1:1592]

In [18]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((459, 1410)), columns = list(range(1, 1411)), index = list(range(1, 460)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    convertMatriksRating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

In [19]:
calTest = ConvertRatingMatriks(testdata)
calTest

,1,2,3,4,5,6,7,8,9,10,...,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410
1,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
def ConvertGTtoList(gt_df):
    groundTruth = []
    # Iterasi untuk setiap user_id di dalam gt_df
    for user_id in gt_df.index:
        # Ambil semua rating user tersebut (baris)
        user_ratings = gt_df.loc[user_id]
        # Ambil item_id yang punya rating > 0 (relevan), ambil nama item koncert ke list
        relevant_items = user_ratings[user_ratings > 0].index.astype(float).tolist()
        # tambahkan ke dalam list groundTruth
        groundTruth.append(relevant_items)
    return groundTruth

In [21]:
rating_Matriks = ConvertRatingMatriks(testdata)
print(rating_Matriks.shape)
# rating_Matriks

# dataset 
allItem = rating_Matriks.columns
alluser = rating_Matriks.index
print(f"all item {len(allItem)}")
itemRated = set(rating_Matriks.columns[rating_Matriks.sum(axis=0) > 0])
print(f"item rated {len(itemRated)}")
userRated = set(rating_Matriks.index[rating_Matriks.sum(axis=1) > 0])
print(f"user rated2 {len(userRated)}")

# item yang tidak pernah di rating
itemNotRated = set(allItem) - itemRated
print(f"item not rated {len(itemNotRated)}")
itemNotRated2 = set(alluser) - userRated
print(f"user not rated2 {len(itemNotRated2)}")

(459, 1410)
all item 1410
item rated 1323
user rated2 457
item not rated 87
user not rated2 2


In [22]:
# toy data ndcg
groundTruthFold1 = [[1, 2, 3, 4, 11], [1, 12, 4, 7], [4, 11, 12, 32], [2, 3, 5, 10]]
groundTruthFold2 = [[3, 5, 7, 10, 11], [2, 3, 5, 11], [1, 10, 11, 12], [4, 7, 12, 21]]
TopNrekFold1 = [[1, 5, 2, 3, 15], [1,4,6,2], [11, 4, 32, 1], [2, 4, 10, 45]]
TopNrekFold2 = [[2, 3, 4, 15, 5], [3,2,12,42], [1, 12, 2, 22], [4, 7, 12, 22]]


In [23]:
groundTruth1 = [1, 2, 3, 4]
TopNrek1 = [1, 5, 2, 3, 15]

In [24]:
print(TopNrek1[:3])
print(TopNrek1[:3][-1] in groundTruth1)

[1, 5, 2]
True


## DCG


$$  DCG(GT_u, TopN_u, N) = \sum_{n=1}^{N} \frac{1}{log_2(1+n)}  \cdot \parallel (TopN_u(n) \in  GT_u) $$

In [25]:
# dcg per user
def EvaluasiDCG(groundTruth, TopNRekomendasi, N):
    """
    Fungsi untuk menghitung DCG (Discounted Cumulative Gain) untuk rekomendasi item
    """
    rumusIDCG = np.array([(1 / np.log2( 1 + x)) for x in range(1, N+1)])
    cekGT = np.array([1 if tp in groundTruth else 0 for tp in TopNRekomendasi[:N]])
    return np.sum(rumusIDCG* cekGT)

In [26]:
# def dcgN(groundTruth, TopNrek, N):
#     totalDCG = 0
#     for i in range(len(groundTruth)):
#         dcgN = EvaluasiDCG(groundTruth[i], TopNrek[i], N)
#         totalDCG += dcgN
#         print(f"DCG untuk user ke-{i+1}: {dcgN}")
#     avgDCG = totalDCG / len(groundTruth)
#     # avgDCG = totalDCG
#     print(f"\nRata-rata DCG: {avgDCG}")
#     return avgDCG

In [27]:
import pandas as pd

def dcgN2(groundTruth, TopNrek, max_N):
    dcg_matrix = []  # Menyimpan DCG tiap user untuk N = 1 sampai max_N

    for i in range(len(groundTruth)):
        user_dcg = []
        print(f"\nUser ke-{i+1}:")
        for N in range(1, max_N + 1):
            dcg_value = EvaluasiDCG(groundTruth[i], TopNrek[i], N)
            user_dcg.append(dcg_value)
            print(f"  DCG@{N}: {dcg_value}")
        dcg_matrix.append(user_dcg)

    # Buat DataFrame
    columns = [f"DCG@{N}" for N in range(1, max_N + 1)]
    df_dcg = pd.DataFrame(dcg_matrix, columns=columns)
    df_dcg.index = [f"User-{i+1}" for i in range(len(groundTruth))]

    # Tambahkan rata-rata DCG@N
    avg_row = df_dcg.mean()
    df_dcg.loc["Rata-rata"] = avg_row

    # print("\nDataFrame DCG per user:")
    # print(df_dcg)

    return df_dcg


In [28]:

calDCGN = dcgN2(groundTruthFold1, TopNrekFold1, 4)
calDCGN


User ke-1:
  DCG@1: 1.0
  DCG@2: 1.0
  DCG@3: 1.5
  DCG@4: 1.9306765580733931

User ke-2:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 1.6309297535714575
  DCG@4: 1.6309297535714575

User ke-3:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 2.1309297535714578
  DCG@4: 2.1309297535714578

User ke-4:
  DCG@1: 1.0
  DCG@2: 1.0
  DCG@3: 1.5
  DCG@4: 1.5


,DCG@1,DCG@2,DCG@3,DCG@4
User-1,1.0,1.000000,1.500000,1.930677
User-2,1.0,1.630930,1.630930,1.630930
User-3,1.0,1.630930,2.130930,2.130930
User-4,1.0,1.000000,1.500000,1.500000
Rata-rata,1.0,1.315465,1.690465,1.798134


In [29]:
# list_gt = [groundTruthFold1, groundTruthFold2]
# list_top = [TopNrekFold1, TopNrekFold2]

# all_df = []

# for gt, top in zip(list_gt, list_top):
#     df = dcgN2(gt, top, max_N=4)
#     df_clean = df.drop("Rata-rata")
#     all_df.append(df_clean)

# df_all = pd.concat(all_df)
# avg_all_folds = df_all.mean().to_frame().T
# avg_all_folds.index = ["Rata-rata Semua Fold"]

# df_final = pd.concat([df_all, avg_all_folds])
# print(df_final)


In [44]:
dfFold1 = dcgN2(groundTruthFold1, TopNrekFold1, 4)
dfFold2 =  dcgN2(groundTruthFold2, TopNrekFold2, 4)

# hapus baris rata-rata
dfCleanRata1 = dfFold1.drop('Rata-rata')
dfCleanRata2 = dfFold2.drop('Rata-rata')

# gabungkan semua user dari semua fold

df_all = pd.concat([dfCleanRata1, dfCleanRata2])

# hitung rata-rata seluruh fold
rata2fold = df_all.mean().to_frame().T
rata2fold.index = ["rata-rata semua fold"]

dfFinalFold = pd.concat([df_all, rata2fold])
dfFinalFold


User ke-1:
  DCG@1: 1.0
  DCG@2: 1.0
  DCG@3: 1.5
  DCG@4: 1.9306765580733931

User ke-2:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 1.6309297535714575
  DCG@4: 1.6309297535714575

User ke-3:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 2.1309297535714578
  DCG@4: 2.1309297535714578

User ke-4:
  DCG@1: 1.0
  DCG@2: 1.0
  DCG@3: 1.5
  DCG@4: 1.5

User ke-1:
  DCG@1: 0.0
  DCG@2: 0.6309297535714575
  DCG@3: 0.6309297535714575
  DCG@4: 0.6309297535714575

User ke-2:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 1.6309297535714575
  DCG@4: 1.6309297535714575

User ke-3:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 1.6309297535714575
  DCG@4: 1.6309297535714575

User ke-4:
  DCG@1: 1.0
  DCG@2: 1.6309297535714575
  DCG@3: 2.1309297535714578
  DCG@4: 2.1309297535714578


,DCG@1,DCG@2,DCG@3,DCG@4
User-1,1.000,1.000000,1.500000,1.930677
User-2,1.000,1.630930,1.630930,1.630930
User-3,1.000,1.630930,2.130930,2.130930
User-4,1.000,1.000000,1.500000,1.500000
User-1,0.000,0.630930,0.630930,0.630930
User-2,1.000,1.630930,1.630930,1.630930
User-3,1.000,1.630930,1.630930,1.630930
User-4,1.000,1.630930,2.130930,2.130930
rata-rata semua fold,0.875,1.348197,1.598197,1.652032


In [31]:
calDCG = EvaluasiDCG(groundTruth1, TopNrek1, 5)
calDCG

np.float64(1.9306765580733931)

## IDCG


$$ IDCG(N) = \sum_{n=1}^{N} \frac{1}{log_2(1+n)}  $$

In [32]:
def IdealDCG(N):
    """
    Fungsi untuk menghitung IDCG (Ideal Discounted Cumulative Gain) untuk rekomendasi item
    """
    rumusIDCG = np.sum(np.array([(1 / np.log2( 1 + x)) for x in range(1, N+1)]))
    # print(rumusIDCG)
    return rumusIDCG

In [33]:
calIdcf = IdealDCG(20)
calIdcf

np.float64(7.040268381923513)

## NDCG

$$ NDCG(GT_u, TopN_u, N) = \frac {DCG(GT_u, TopN_u, N)}{IDCG(N)} $$ 

In [34]:
def EvaluasiNDCG(groundTruth, TopNRekomendasi, N):
    """
    Fungsi untuk menghitung NDCG (Normalized Discounted Cumulative Gain) untuk rekomendasi item
    """
    evaluasiNDCG = EvaluasiDCG(groundTruth, TopNRekomendasi, N) / IdealDCG(N)
    return evaluasiNDCG

In [35]:
# def ndcgN(groundTruth, TopNrek, N):
#     totalNDCG = 0
#     for i in range(len(groundTruth)):
#         ndcgN = EvaluasiNDCG(groundTruth[i], TopNrek[i], N)
#         totalNDCG += ndcgN
#         print(f"nDCG untuk data ke-{i+1}: {ndcgN}")
#     avgNDCG = totalNDCG / len(groundTruth)
#     print(f"\nRata-rata NDCG: {avgNDCG}")
#     return avgNDCG

In [36]:
def NDCGN(groundTruth, TopNrek, max_N):
    ndcg_matrix = []  # Menyimpan DCG tiap user untuk N = 1 sampai max_N

    for i in range(len(groundTruth)):
        user_ndcg = []
        print(f"\nUser ke-{i+1}:")
        for N in range(1, max_N + 1):
            ndcg_value = EvaluasiNDCG(groundTruth[i], TopNrek[i], N)
            user_ndcg.append(ndcg_value)
            print(f"  NDCG@{N}: {ndcg_value}")
        ndcg_matrix.append(user_ndcg)

    # Buat DataFrame
    columns = [f"NDCG@{N}" for N in range(1, max_N + 1)]
    df_ndcg = pd.DataFrame(ndcg_matrix, columns=columns)
    df_ndcg.index = [f"User-{i+1}" for i in range(len(groundTruth))]

    # Tambahkan rata-rata DCG@N
    avg_row = df_ndcg.mean()
    df_ndcg.loc["Rata-rata"] = avg_row

    # print("\nDataFrame DCG per user:")
    # print(df_dcg)

    return df_ndcg


In [37]:
calNDCGN = NDCGN(groundTruthFold1, TopNrekFold1, 4)
calNDCGN


User ke-1:
  NDCG@1: 1.0
  NDCG@2: 0.6131471927654584
  NDCG@3: 0.7039180890341347
  NDCG@4: 0.75369761125927

User ke-2:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 0.7653606369886217
  NDCG@4: 0.6366824387328317

User ke-3:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 1.0
  NDCG@4: 0.8318724637288826

User ke-4:
  NDCG@1: 1.0
  NDCG@2: 0.6131471927654584
  NDCG@3: 0.7039180890341347
  NDCG@4: 0.5855700749881525


,NDCG@1,NDCG@2,NDCG@3,NDCG@4
User-1,1.0,0.613147,0.703918,0.753698
User-2,1.0,1.000000,0.765361,0.636682
User-3,1.0,1.000000,1.000000,0.831872
User-4,1.0,0.613147,0.703918,0.585570
Rata-rata,1.0,0.806574,0.793299,0.701956


In [45]:
dfFold1 = NDCGN(groundTruthFold1, TopNrekFold1, 4)
dfFold2 =  NDCGN(groundTruthFold2, TopNrekFold2, 4)

# hapus baris rata-rata
dfCleanRata1 = dfFold1.drop('Rata-rata')
dfCleanRata2 = dfFold2.drop('Rata-rata')

# gabungkan semua user dari semua fold

df_all = pd.concat([dfCleanRata1, dfCleanRata2])

# hitung rata-rata seluruh fold
rata2fold = df_all.mean().to_frame().T
rata2fold.index = ["rata-rata semua fold"]

dfFinalFold = pd.concat([df_all, rata2fold])
dfFinalFold


User ke-1:
  NDCG@1: 1.0
  NDCG@2: 0.6131471927654584
  NDCG@3: 0.7039180890341347
  NDCG@4: 0.75369761125927

User ke-2:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 0.7653606369886217
  NDCG@4: 0.6366824387328317

User ke-3:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 1.0
  NDCG@4: 0.8318724637288826

User ke-4:
  NDCG@1: 1.0
  NDCG@2: 0.6131471927654584
  NDCG@3: 0.7039180890341347
  NDCG@4: 0.5855700749881525

User ke-1:
  NDCG@1: 0.0
  NDCG@2: 0.38685280723454163
  NDCG@3: 0.2960819109658652
  NDCG@4: 0.24630238874073

User ke-2:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 0.7653606369886217
  NDCG@4: 0.6366824387328317

User ke-3:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 0.7653606369886217
  NDCG@4: 0.6366824387328317

User ke-4:
  NDCG@1: 1.0
  NDCG@2: 1.0
  NDCG@3: 1.0
  NDCG@4: 0.8318724637288826


,NDCG@1,NDCG@2,NDCG@3,NDCG@4
User-1,1.000,0.613147,0.703918,0.753698
User-2,1.000,1.000000,0.765361,0.636682
User-3,1.000,1.000000,1.000000,0.831872
User-4,1.000,0.613147,0.703918,0.585570
User-1,0.000,0.386853,0.296082,0.246302
User-2,1.000,1.000000,0.765361,0.636682
User-3,1.000,1.000000,0.765361,0.636682
User-4,1.000,1.000000,1.000000,0.831872
rata-rata semua fold,0.875,0.826643,0.750000,0.644920


In [39]:
calNDCG = EvaluasiNDCG(groundTruth1, TopNrek1, 4)
calNDCG

np.float64(0.75369761125927)

## LOPP TEST REAL DATA

In [40]:
openTopN = joblib.load("../case/topN/testTopn1.joblib")
openTopN2 = joblib.load("../case/topN/testTopn2.joblib")

# jadikan data frame
print(len(openTopN))

459


In [41]:
convertGt = ConvertGTtoList(calTest)
print(len(convertGt))

459


In [42]:
# calDCGN = dcgN(convertGt, openTopN2, 20)


In [43]:
calNDCGN = ndcgN(convertGt, openTopN2, 20)
    

NameError: name 'ndcgN' is not defined

In [ ]:
# ambil rata2 dcg 

hitungNDCG = (calDCGN /calIdcf)
print(f"fold=1 {hitungNDCG}")

fold=1              DCG@1     DCG@2     DCG@3     DCG@4
User-1     0.14204  0.142040  0.213060  0.274233
User-2     0.14204  0.231657  0.231657  0.231657
User-3     0.14204  0.231657  0.302677  0.302677
User-4     0.14204  0.142040  0.213060  0.213060
Rata-rata  0.14204  0.186849  0.240114  0.255407
